# Define parameters

In [1]:
import subprocess
import os

is_front_camera_training = False  # Change it to False if training for down cmaera.

In [2]:
################################# OS PARAMETERS ##################################
os_name = os.name
# Windows = nt, [Linux, Apple] = posix. 
os_path = "\\" if os_name == "nt" else "/"  
##################################################################################
############################## ROBOFLOW  PARAMETERS ##############################
roboflow_api_key = "MJQUATZvpcKoBxRjLuXx"
roboflow_workspace_name = "auv2024"
dataset_version_download = "yolov8"
if is_front_camera_training:
     roboflow_project_name = "front-camera-sim"
     roboflow_project_version = 6
else:
     roboflow_project_name = "down-camera-sim"
     roboflow_project_version = 1
##################################################################################
############################## TRAINING  PARAMETERS ##############################
if is_front_camera_training:
     target_classes = ["bouy", "gate", "octagon-table"]
     model_save_filename = "best_AUV_sim_front_camera_model.pt"
else: 
     target_classes = ["bin", "lane-marker", "octagon-table"]
     model_save_filename = "best_AUV_sim_down_camera_model.pt"
model_name = "yolov8n.pt"
train_test_val_split = (0.7, 0.2, 0.1)
epoch_increments = 200
batch_size = -1 # Auto Mode (60% GPU Memory): Use batch=-1 to automatically 
                # adjust batch size for approximately 60% CUDA memory 
                # utilization.
workers = 1
cache = False
pretrained = True
hsv_h = 0.001
hsv_s = 0.1
hsv_v = 0.1
translate = 0.1
scale = 0.2
fliplr = 0.2
mosaic = 0.5
erasing = 0.1
crop_fraction = 0.5
##################################################################################
######################### CUSTOM AUGMENTATION PARAMETERS #########################
colorAugmentProb = 0.3
noiseAugmentProb = 0.3
resolutionAugmentProb = 0.3
contrastAugmentProb = 0.3
blurAugmentProb = 0.3
brightnessAugmentProb = 0.3
##################################################################################
########################## YOLO AUGMENTATION PARAMETERS ##########################
# Currently not being used.
degrees = 360
flipud = 0.5
fliplr = 0.5
max_perspective_change = 0.001
max_translate = 0.1
max_scale_change = 0.3
mosaic = 0.5
mixup = 0.5
##################################################################################

# Implementation

## Setup python dependencies 

In [3]:
packages = [
     "roboflow", 
     "albumentations", 
     "opencv-python", 
     "ultralytics"
]

try:
    for package in packages:
        subprocess.check_call(["pip", "install", package])
    print("Packages installed successfully.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}.")

Packages installed successfully.


In [4]:
import shutil
import cv2
import albumentations as A
import copy
import random
import numpy as np
from ultralytics import YOLO
import torch
from roboflow import Roboflow

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.10 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


In [5]:
!mkdir data
!mkdir data\augmented
!mkdir data\augmented\train
!mkdir data\augmented\test
!mkdir data\augmented\val
!mkdir data\augmented\train\images
!mkdir data\augmented\test\images
!mkdir data\augmented\val\images
!mkdir data\augmented\train\labels
!mkdir data\augmented\test\labels
!mkdir data\augmented\val\labels
!mkdir data\raw
!mkdir data\raw\images
!mkdir data\raw\labels

## Define YOLO classes

In [6]:
data_folder_absolute_path = os.path.abspath("data")
print(f"Data absolute path: {data_folder_absolute_path}.")

with open("data.yaml", "w+") as f:
    f.write(f"train: {data_folder_absolute_path}{os_path}augmented{os_path}train{os_path}images\n")
    f.write(f"test: {data_folder_absolute_path}{os_path}augmented{os_path}test{os_path}images\n")
    f.write(f"val: {data_folder_absolute_path}{os_path}augmented{os_path}val{os_path}images\n")
    f.write(f"nc: {len(target_classes)}\n")
    f.write(f"names: {target_classes}")

Data absolute path: c:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data.


## Define augmentation functions

In [7]:
# Given a list of samples, make two copies of each sample that are darker/brighter to simulate differently lit environments.
def brightnessAugment(images):
    out = []
    for image in images:
        transform = A.Compose([A.augmentations.transforms.ColorJitter(brightness=(1.5, 1.5), contrast=0, saturation=0, hue=0, always_apply=True)])
        bright_img = transform(image=image)["image"]
        transform = A.Compose([A.augmentations.transforms.ColorJitter(brightness=(0.5, 0.5), contrast=0, saturation=0, hue=0, always_apply=True)])
        dark_img = transform(image=image)["image"]
        out.append(bright_img)
        out.append(dark_img)
    return out

# Given a list of samples, make a copy of each sample but more blurred to simulate objects out of focus, dirty lenses, and backscattering.
def blurAugment(images):
    out = []
    for image in images:
        ksize = (8, 8) # lower to lower blur
        blurred_img = cv2.blur(image, ksize)
        out.append(blurred_img)
    return out

# Given a list of samples, make a copy of each sample but with a lower contrast image to simulate backscattering and over/under-exposure.
def contrastAugment(images):
    out = []
    for image in images:
        transform = A.Compose([A.augmentations.transforms.ColorJitter (brightness=0, contrast=(0.5, 0.5), saturation=0, hue=0, always_apply=True)])
        decontrasted_img = transform(image=image)["image"]
        out.append(decontrasted_img)
    return out

# Given a list of samples, make a copy of each sample but with camera noise added to the image to simulate different camera feeds.
def noiseAugment(images):
    out = []
    for image in images:
        transform = A.Compose([A.augmentations.transforms.ISONoise(color_shift=(0.1, 0.1), intensity=(0.5, 0.5), always_apply=True)])
        noisy_img = transform(image=image)["image"]
        out.append(noisy_img)
    return out

# Given a list of samples, make a copy of each sample but with the image downscaled (lower resolution of image) to simulate lower quality cameras/images.
def resolutionAugment(images):
    out = []
    for image in images:
        transform = A.Compose([A.augmentations.transforms.Downscale(scale_min=0.2, scale_max=0.2, always_apply=True)])
        low_res_img = transform(image=image)["image"]
        out.append(low_res_img)
    return out

# Increase intensity of blues in given image.
def make_bluer(img, color_shift_intensity):
    img_b, img_g, img_r = cv2.split(img) # Split by channel.
    img_b = np.uint16(img_b)
    img_b += color_shift_intensity
    np.clip(img_b, 0, 255, out=img_b)
    img_b = np.uint8(img_b)
    img = cv2.merge((img_b, img_g, img_r)) # Merge adjusted channels.
    del img_b
    del img_g
    del img_r
    return img

# Increase intensity of greens in given image.
def make_greener(img, color_shift_intensity):
    img_b, img_g, img_r = cv2.split(img) # Split by channel.
    img_g = np.uint16(img_g)
    img_g += color_shift_intensity
    np.clip(img_g, 0, 255, out=img_g)
    img_g = np.uint8(img_g)
    img = cv2.merge((img_b, img_g, img_r)) # Merge adjusted channels.
    del img_b
    del img_g
    del img_r
    return img

# Given a list of samples, make two copies of each sample (one bluer, one greener) to simulate different pools + color attenuation.
def colorAugment(images):
    out = []
    color_shift_intensity = int(255*0.05)
    for image in images:
        blue_img = make_bluer(image, color_shift_intensity)
        green_img = make_greener(image, color_shift_intensity)
        out.append(blue_img)
        out.append(green_img)
    return out

# Given a single image and augmentation function, displays the image before and images after augmentation.
def visualizeAugmentation(img, aug):
    # Show original image.
    cv2.imshow("og", img)
    cv2.waitKey(0)
    # Show all augmented images.
    for augmented in aug([(img, "")])[1:]:
        cv2.imshow("augmented", augmented[0])
        cv2.waitKey(0)

In [8]:
def get_file_names(source_folder):
    label_filenames = []
    img_filenames = [f for f in os.listdir(source_folder + f"{os_path}images") if os.path.isfile(os.path.join(source_folder + f"{os_path}images", f))]
    for img_filename in img_filenames:
        label_filenames.append(os.path.splitext(img_filename)[0] + ".txt")

    return np.array(img_filenames), np.array(label_filenames)

def split_file_names(images, labels, splits):
    perm = np.random.permutation(len(images))
    images = images[perm]
    labels = labels[perm]
    splits = [int(len(images)*s) for s in splits]
    train_images = images[:splits[0]]
    train_labels = labels[:splits[0]]
    val_images = images[splits[0]: splits[0] + splits[1]]
    val_labels = labels[splits[0]: splits[0] + splits[1]]
    test_images = images[splits[0] + splits[1]:]
    test_labels = labels[splits[0] + splits[1]:]
    return train_images, train_labels, val_images, val_labels, test_images, test_labels

def get_augs(img_filename, source_folder):
    img = cv2.imread(source_folder + f"{os_path}images{os_path}" + img_filename)
    augs = [img]
    if(np.random.rand() < colorAugmentProb):
        augs = augs + colorAugment(augs)
    if(np.random.rand() < noiseAugmentProb):
        augs = augs + noiseAugment(augs)
    if(np.random.rand() < resolutionAugmentProb):
        augs = augs + resolutionAugment(augs)
    if(np.random.rand() < contrastAugmentProb):
        augs = augs + contrastAugment(augs)
    if(np.random.rand() < blurAugmentProb):
        augs = augs + blurAugment(augs)
    if(np.random.rand() < brightnessAugmentProb):
        augs = augs + brightnessAugment(augs)
    return augs

def do_augs_and_export(img_filenames, label_filenames, source_folder, output_folder):
    name_num = 1
    for (img_filename, label_filename) in zip(img_filenames, label_filenames):
        augs = get_augs(img_filename, source_folder)
        with open(source_folder + f"{os_path}labels{os_path}" + label_filename) as f:
            #build array of bounding boxes (each line its own element)
            bounding_boxes = f.read()
        for aug in augs:
            cv2.imwrite(output_folder + f"{os_path}images{os_path}img" + str(name_num) + ".png", aug)
            with open(output_folder + f"{os_path}labels{os_path}img" + str(name_num) + ".txt", "w+") as f:
                f.write(bounding_boxes)
            name_num+=1

## Download dataset from Roboflow

In [9]:
rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace(roboflow_workspace_name).project(roboflow_project_name)
version = project.version(roboflow_project_version)
dataset = version.download(dataset_version_download)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.35, to fix: `pip install ultralytics==8.0.196`
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to down-camera-sim-1 in yolov8:: 100%|██████████| 226/226 [00:00<00:00, 3302.65it/s]


In [10]:
folder_name = "{}-{}".format(roboflow_project_name, roboflow_project_version)
roboflow_folder_path = os.path.abspath(folder_name)
data_folder_path = os.path.abspath("data")

print(f"Roboflow folder name: {folder_name}.")

def copy_all_files(source, destination):
     try:
          # Ensure destination folder exists.
          os.makedirs(destination, exist_ok=True)
          # Copy all files from source to destination.
          for filename in os.listdir(source):
               source_file = os.path.join(source, filename)
               if os.path.isfile(source_file):
                    destination_file = os.path.join(destination, filename)
                    shutil.move(source_file, destination_file)
     except Exception as e:
          print(f"Error copying files: {e}.")

# Copy files from roboflow to data folder.
for folder in ["train", "valid", "test"]:
     print(f"Moving {folder} folder...")
     copy_all_files(os.path.join(roboflow_folder_path, folder, "images"), 
                    os.path.join(data_folder_path, "raw", "images"))
     copy_all_files(os.path.join(roboflow_folder_path, folder, "labels"), 
                    os.path.join(data_folder_path, "raw", "labels"))
     print(f"Moving {folder} folder completed.")



# Optional: Remove roboflow folder after copying files.
try:
     shutil.rmtree(folder_name)
     print(f"Removed folder: {folder_name}.")
except Exception as e:
     print(f"Error removing folder: {e}.")

Roboflow folder name: down-camera-sim-1.
Moving train folder...
Moving train folder completed.
Moving valid folder...
Moving valid folder completed.
Moving test folder...
Moving test folder completed.
Removed folder: down-camera-sim-1.


## Augment data, split into train/test/val

In [11]:
%%capture
out_folder = f"data{os_path}augmented"
in_folder = f"data{os_path}raw"
train = f"{os_path}train"
val = f"{os_path}val"
test = f"{os_path}test"

raw_image_names, raw_label_names = get_file_names(in_folder)
num_raw_samples = len(raw_image_names)
train_images, train_labels, val_images, val_labels, test_images, test_labels = split_file_names(raw_image_names, raw_label_names, train_test_val_split)
do_augs_and_export(train_images, train_labels, in_folder, out_folder + train)
do_augs_and_export(val_images, val_labels, in_folder, out_folder + val)
do_augs_and_export(test_images, test_labels, in_folder, out_folder + test)

In [12]:
augmented_train_img_names, _ = get_file_names(out_folder + train)
augmented_test_img_names, _ = get_file_names(out_folder + test)
augmented_val_img_names, _ = get_file_names(out_folder + val)

num_augmented_samples = len(augmented_train_img_names) + len(augmented_test_img_names) + len(augmented_val_img_names)

print(f"Augmentation completed: went from {num_raw_samples} samples to {num_augmented_samples} after augmentation.")

Augmentation completed: went from 107 samples to 728 after augmentation.


# Check CUDA dependencies and start training

In [13]:
if not (torch.cuda.is_available() and torch.cuda.device_count()):
    raise RuntimeError("CUDA is NOT available. Please ensure that your system has a compatible GPU and the necessary drivers are installed.")

In [14]:
runs_folder = "runs"
detect_folder = "detect"

if os.path.exists(runs_folder):
    shutil.rmtree(runs_folder)

os.makedirs(os.path.join(runs_folder, detect_folder), exist_ok=True)

In [15]:
data_yaml_file_absolute_path = os.path.abspath("data.yaml")

model = YOLO("yolov8n.pt") # load a pretrained model.

# Start the training process.
while True:
    try:
        model.train(
            data=data_yaml_file_absolute_path,
            epochs=epoch_increments,
            batch=batch_size,
            pretrained=pretrained,
            task="detect",
            cache=cache,
            workers=workers,
            hsv_h=hsv_h,
            hsv_s=hsv_s,
            hsv_v=hsv_v,
            translate=translate,
            scale=scale,
            fliplr=fliplr,
            mosaic=mosaic,
            erasing=erasing,
            crop_fraction=crop_fraction
        )
        shutil.copyfile(f"runs{os_path}detect{os_path}train{os_path}weights{os_path}best.pt", model_save_filename)
    except RuntimeError as e:
        print(f"Caught a RuntimeError: {e}.")
        break  # Break out of the loop if an error occurs to prevent infinite loop.

New https://pypi.org/project/ultralytics/8.2.45 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.35  Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=c:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data.yaml, epochs=200, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False,

train: Scanning C:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data\augmented\train\labels... 662 images, 186 backgrounds, 0 corrupt: 100%|██████████| 662/662 [00:01<00:00, 654.64it/s]


train: New cache created: C:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data\augmented\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data\augmented\val\labels... 371 images, 16 backgrounds, 0 corrupt: 100%|██████████| 371/371 [00:00<00:00, 763.82it/s]

val: New cache created: C:\Users\poten\AUV-2024\catkin_ws\src\vision\model_pipeline\data\augmented\val\labels.cache


...

    188/200      5.18G     0.2209     0.1806     0.7981         25        640: 100%|██████████| 18/18 [00:05<00:00,  3.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  3.04it/s]

                   all        371        728      0.972      0.937      0.967       0.83



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    189/200      5.16G     0.2457     0.1874     0.8038         58        640:   6%|▌         | 1/18 [00:00<00:05,  2.91it/s]


KeyboardInterrupt: 